In [1]:
# import load_dotenv
from dotenv import load_dotenv

# load env
load_dotenv()

True

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from langgraph.prebuilt import create_react_agent

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model="anthropic:claude-3-7-sonnet-latest",  
    tools=[get_weather],  
    prompt="You are a helpful assistant"  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='c7cfde00-d1a6-4435-89fa-cdee62810891'),
  AIMessage(content=[{'text': 'I can help you check the weather in San Francisco. Let me get that information for you.', 'type': 'text'}, {'id': 'toolu_01GFnwDt5n3Wic2ZDkvHZX2Q', 'input': {'city': 'San Francisco'}, 'name': 'get_weather', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01QV7SadAG8gqxQ2tJvoLb7b', 'model': 'claude-3-7-sonnet-latest', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 386, 'output_tokens': 74, 'server_tool_use': None}, 'model_name': 'claude-3-7-sonnet-latest'}, id='run--f750a6ba-6135-4411-ad21-99fb8519580f-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'toolu_01GFnwDt5n3Wic2ZDkvHZX2Q', 'type': 'tool_call'}], usage_metadata={'input_tokens': 386, 'output_tokens':

In [13]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain_anthropic import ChatAnthropic
from datetime import datetime

# Define the tool
def get_current_time() -> str:
    """Returns the current time in 12-hour format with AM/PM indicator.
    
    Returns:
        str: The current time formatted as HH:MM AM/PM
    """
    return datetime.now().strftime("%I:%M %p")

# Initialize the model
model = ChatAnthropic(model="claude-3-7-sonnet-latest")

# Create the agent with the tool and system prompt
agent = create_react_agent(
    model=model,
    tools=[get_current_time],
    prompt="Be as concise as possible, reply with one sentence."
)

# Set up in-memory checkpointer for conversation history
checkpointer = InMemorySaver()

# Add checkpointer to the agent
agent_with_memory = create_react_agent(
    model=model,
    tools=[get_current_time],
    prompt="Be as concise as possible, reply with one sentence.",
    checkpointer=checkpointer
)

# Main interaction loop
config = {"configurable": {"thread_id": "conversation-1"}}

while True:
    current_message = input('You: ')
    if current_message == 'quit':
        break
    
    # Invoke the agent with the current message
    result = agent_with_memory.invoke(
        {"messages": [{"role": "user", "content": current_message}]},
        config
    )
    
    # Extract and print the assistant's response
    assistant_message = result["messages"][-1]
    print(f'Agent respose: {assistant_message.content}')
    
    # Print message history and result data for debugging
    print(f"Result data: {result}")

You:  My name is Juan. What's the time right now?


Agent respose: Hello Juan, the current time is 05:35 PM.
Result data: {'messages': [HumanMessage(content="My name is Juan. What's the time right now?", additional_kwargs={}, response_metadata={}, id='07f53316-746c-49cc-9818-78b1798b391b'), AIMessage(content=[{'id': 'toolu_013EfXSrL986E1X34XbQPtSC', 'input': {}, 'name': 'get_current_time', 'type': 'tool_use'}], additional_kwargs={}, response_metadata={'id': 'msg_01LYXjXAR3b8nVwop5JUSzsM', 'model': 'claude-3-7-sonnet-latest', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 419, 'output_tokens': 38, 'server_tool_use': None}, 'model_name': 'claude-3-7-sonnet-latest'}, id='run--4d69c9b3-cccd-4214-8a0c-764046b62e5b-0', tool_calls=[{'name': 'get_current_time', 'args': {}, 'id': 'toolu_013EfXSrL986E1X34XbQPtSC', 'type': 'tool_call'}], usage_metadata={'input_tokens': 419, 'output_tokens': 38, 'total_tokens': 457, 'input_token_details': {'cache_read': 0, 

You:  quit
